In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import re
import requests
import holoviews as hv
import matplotlib.pyplot as plt
import pint
import nd2reader
import tifffile
import scipy.stats
import skimage
import skimage.filters

In [ ]:
%load_ext autoreload
%autoreload 2
import simulation

In [ ]:
# u = simulation.ureg
class ObjProxy(object):
    def __init__(self, module_name, attr_name):
        self.__module_name = module_name
        self.__attr_name = attr_name

    def __getattr__(self, name):
        return getattr(getattr(globals()[self.__module_name], self.__attr_name), name)


u = ObjProxy("simulation", "ureg")

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
excitation_bins = np.linspace(300, 850, 200)
excitation_bin_size = (excitation_bins[-1] - excitation_bins[0]) / len(excitation_bins)
emission_bins = np.linspace(300, 850, 200)  # excitation_bins
emission_bin_size = (emission_bins[-1] - emission_bins[0]) / len(emission_bins)

# FP spectra

In [ ]:
spectra_urls = [
    "https://www.fpbase.org/spectra_csv/?q=79,80,158",
    "https://www.fpbase.org/spectra_csv/?q=1451,1450",
    "https://www.fpbase.org/spectra_csv/?q=119,120",
    "https://www.fpbase.org/spectra_csv/?q=121,122",
]

In [ ]:
spectra = {
    name: simulation.interpolate_dataframe(spectrum, emission_bins)
    for name, spectrum in simulation.import_fpbase_spectra(spectra_urls).items()
}

In [ ]:
fp_names = list(spectra.keys())
fp_excitation_spectra = np.stack(
    [np.zeros(len(excitation_bins)), *[spectra[name]["ex"].values for name in fp_names]]
)
seesaw_amounts = np.linspace(-0.5, 1, len(excitation_bins))[np.newaxis, :]
fp_emission_spectra = np.stack(
    [
        np.zeros((len(emission_bins), len(excitation_bins))),
        *[
            simulation.seesaw_spectrum(
                spectra[name]["em"].values[:, np.newaxis], seesaw_amounts
            )
            for name in fp_names
        ],
    ]
)

In [ ]:
plt.imshow(fp_emission_spectra[3])

# Excitation line

In [ ]:
line_img = simulation.draw_excitation_line(
    np.linspace(0.6, 1.3, len(excitation_bins)) * u.mm,
    0.3 * u.um,
    0 * u.um,
    1,
    p_vertical=3,
    p_horizontal=6,
    height_px=60,
    width_px=6500,
)

In [ ]:
line_img.nbytes / 1e6

In [ ]:
line_img[0].plot()

In [ ]:
line_img[-1].plot()

# Sample

In [ ]:
# nd2 = nd2reader.ND2Reader("ClpP_mEGFP_100x.nd2")
# nd2._parser._raw_metadata.z_levels = [0]
# sample_img = nd2.get_frame_2D(v=0, c=1)  # [:,500:1000]
# sample_img = image_to_xarray(sample_img, nd2.metadata["pixel_microns"])

In [ ]:
sample_img = tifffile.imread("t000001xy14c2.tif")
scale = 4.25 / 20
sample_img = sample_img / sample_img.max()
sample_img = simulation.image_to_xarray(sample_img, scale)

In [ ]:
labels = sample_img.values > skimage.filters.threshold_otsu(sample_img.values)

In [ ]:
num_spectra = 4

In [ ]:
fp_idx = skimage.morphology.label(labels) % num_spectra + (labels != 0)
for i in range(3):
    fp_idx = skimage.morphology.dilation(fp_idx)

In [ ]:
fp_img = simulation.xarray_like(sample_img, fp_idx)

# Regrid FP image to excitation line grid

In [ ]:
shifts = dict(x=500, y=470)

In [ ]:
fp_img_offset = simulation.shift_and_interp(fp_img, line_img, shifts, method="nearest")

In [ ]:
fp_img_offset.plot(aspect=20, size=5)

In [ ]:
sample_img_offset = simulation.shift_and_interp(sample_img, line_img, shifts)

In [ ]:
(fp_img_offset * sample_img_offset).plot(aspect=20, size=5)

# Laser spectrum

In [ ]:
laser_spectrum = scipy.stats.norm.pdf(excitation_bins, 550, 10) * excitation_bin_size
laser_spectrum = xr.DataArray(
    laser_spectrum, coords=dict(ex=excitation_bins), dims=["ex"]
)

In [ ]:
laser_spectrum.plot()

# Laser filter spectrum

# Excitation image

In [ ]:
excitation_img = line_img * laser_spectrum

# Absorption image

In [ ]:
fp_img_offset.astype(np.uint32)

In [ ]:
sample_excitation_img = xr.DataArray(
    np.rollaxis(fp_excitation_spectra[fp_img_offset.astype(np.uint8), :], -1),
    coords=dict(ex=excitation_bins, x=fp_img_offset.x, y=fp_img_offset.y),
    dims=["ex", "y", "x"],
)

In [ ]:
sample_excitation_intensity_img = (sample_excitation_img * sample_img_offset).fillna(0)

In [ ]:
sample_excitation_intensity_img

In [ ]:
sample_excitation_intensity_img[30].plot(aspect=20, size=5)

In [ ]:
sample_excitation_intensity_img[:, 30, 3180:3230].plot(aspect=20, size=5)

In [ ]:
absorption_img = excitation_img * sample_excitation_intensity_img

In [ ]:
absorption_img[60].plot(aspect=20, size=5)

# Emission image

# Reflected emission

# 